# 数据库操作shell命令

### 查看版本号

```
mongo --version
MongoDB shell version v3.6.3
```  

注意: 
1. 这个命令在控制台输入,而不是mongodb交互环境中的命令
2. 一般用于安装好,检查mongodb是否安装成功

### 创建数据库,账户和密码

**注意:**
1. MongoDB中的账号是在某一个库里面进行设置的，也就是帐号密码和具体的数据库绑定,在哪一个库里进行设置，就要在哪一个库里进行验证。
2. 创建用户时，需要指定用户名、用户密码和用户角色，用户角色表示了该用户的权限。

### 创建管理员账户和密码

```
> use admin
switched to db admin
> db.createUser({user:"root",pwd:"root",roles:["userAdminAnyDatabase"]})
Successfully added user: { "user" : "root", "roles" : [ "userAdminAnyDatabase" ] }
> db.auth("root","root123")
1
```
db.auth() 方法返回 1 表示登录成功。接下来我们为指定的数据库创建访问所需的账号。

**说明:**
1. 用户管理员是第一个要创建的用户。在没有创建任何用户之前，你可以随意创建用户；但数据库中一旦有了用户，那么未登录的客户端就没有权限做任何操作了，除非使用db.auth(username, password)方法登录。
2. 用户管理员的角色名叫 userAdminAnyDatabase，这个角色只能在 admin 数据库中创建。

### 创建普通账户和密码

```
> use test
switched to db test
> db.createUser({user:"testuser",pwd:"testpass",roles:["readWrite"]})
Successfully added user: { "user" : "testuser", "roles" : [ "readWrite" ] }
> db.auth("testuser","testpass")
1
```

**说明:**
1. 首先保证你已经以用户管理员的身份登录 admin 数据库。然后用 use 命令切换到目标数据库，
2. 用 db.createUser() 命令来创建用户，其中角色名为 “readWrite”。
3. 普通的数据库用户角色有两种，read 和 readWrite。前者只能读取数据不能修改，后者可以读取和修改。

### 进入和退出mongo的shell环境

1. 进入mongo shell环境:
```
> mongo
MongoDB shell version v3.6.3
connecting to: mongodb://127.0.0.1:27017
MongoDB server version: 3.6.3
```

2. 退出mongo shell环境:
```
> exit
bye
```

### 连接Mongo服务

1. 本地安装mongodb
2. 在终端输入: 
    * 方式一: `mongo ip:port/dbname -u username -p`, 回车然后输入密码,或者`mongo ip:port/dbname -u username -p passwd`,回车
    * 方式二: `mongo -u username -p passwd 192.168.0.197:27017/dbname`, 回车

其中, ip是服务器的ip, port是服务器上mongo指定的端口, dbname是你要访问的数据库, dbname不能少

### 创建管理用户

mongodb没有超级无敌用户root

只有管理用户的用户: userAdminAnyDatabase

### 查看帮助信息

```
// 查看所有数据级别的操作
> db.help()
// 查看集合级别的操作
> db.mycoll.help()
// 列举数据库命令
> db.listCommands()
```

### 显示当前正在使用的数据库

```
> db
test
```

### 显示当前数据库的用户信息

```
> show users
{
        "_id" : "admin.root",
        "user" : "root",
        "db" : "admin",
        "roles" : [
                {
                        "role" : "userAdminAnyDatabase",
                        "db" : "admin"
                }
        ]
}
```

### 显示所有数据库

```
> show dbs
hawk  0.136GB
admin    0.000GB
config   0.000GB
local    0.000GB
testdb   0.001GB
```

### 创建或切换指定数据库

切换到testdb数据库:
```
> use testdb
switched to db testdb
```

### 删除数据库

`db.dropDatabase()`

### 显示当前数据库中所有的集合

```
> show collections
info
users
score
```
或者
`db.getCollectionNames()`

### 查询指定数据库统计信息

```
use dbname
db.stats()
```

### 创建和删除集合

```
> db.createCollection("users")
{ "ok" : 1 }
> db.users.drop()
{ "ok" : 1 }
```

### 查询指定数据库的集合当前可用的存储空间

`db.baseSe.storageSize()`

### 查询指定数据库的集合分配的存储空间

`db.baseSe.totalSize()`

### 数据备份

1. 备份所有数据库
```
mkdir testbak
cd testbak
mongodump
```
说明：默认备份目录及数据文件格式为: `/dump/[databasename]/[collectionname].bson`

2. 备份指定数据库: 
`mongodump -d pagedb`

3. 备份一个数据库中的某个集合: 
`mongodump -d pagedb -c page`


### 数据恢复

1. 恢复全部数据库
```
cd testbak
mongorestore --drop
```
说明：将备份的所有数据库恢复到数据库，--drop指定恢复数据之前删除原来数据库数据，否则会造成回复后的数据中数据重复。

2. 恢复指定数据库的数据
```
cd testbak
mongorestore -d pagedb --drop
```
说明：将备份的pagedb的数据恢复到数据库。

3. 恢复指定数据库的指定集合的数据
```
cd testbak
mongorestore -d pagedb -c page --drop
```
说明：将备份的pagedb的的page集合的数据恢复到数据库。

### 数据导出与导入

1. 向MongoDB导入数据: 
`mongoimport -d pagedb -c page --type csv --headerline --drop < csvORtsvFile.csv`

说明：将文件csvORtsvFile.csv的数据导入到pagedb数据库的page集合中，使用cvs或tsv文件的列名作为集合的列名。

注意: 使用`--headerline`选项时，只支持csv和tsv文件。
`--type`支持的类型有三个：csv、tsv、json
其他各个选项的使用，可以查看帮助：

2. 从MongoDB导出数据: 
`mongoexport -d pagedb -c page -q {} -f _id,title,url,spiderName,pubDate --csv > pages.csv`

说明：将pagedb数据库中page集合的数据导出到pages.csv文件

各选项含义：
`-f` 指定cvs列名为_id,title,url,spiderName,pubDate
`-q` 指定查询条件

其他各个选项的使用，可以查看帮助：